In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

# Build Dataset

In [4]:
# read in all the words
with open("names.txt") as fp:
    words = fp.read().splitlines()
words[:7]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia']

In [8]:
# build a vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [17]:
block_size = 3
X, Y = [], []

for word in words[:5]:
    context = [0] * block_size
    print(word)
    for c in word + '.':
        ix = stoi[c]
        X.append(context)
        Y.append(ix)
        print(f"{''.join([itos[i] for i in context])} -> {c}")
        context =context[1:] + [ix]

X = torch.tensor(X, dtype=int)
Y = torch.tensor(Y, dtype=int)

emma
... -> e
..e -> m
.em -> m
emm -> a
mma -> .
olivia
... -> o
..o -> l
.ol -> i
oli -> v
liv -> i
ivi -> a
via -> .
ava
... -> a
..a -> v
.av -> a
ava -> .
isabella
... -> i
..i -> s
.is -> a
isa -> b
sab -> e
abe -> l
bel -> l
ell -> a
lla -> .
sophia
... -> s
..s -> o
.so -> p
sop -> h
oph -> i
phi -> a
hia -> .


## Embedding Layer

In [ ]:
C = torch.randn((27, 2))
C.shape

torch.Size([27, 2])

In [19]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [24]:
embed = C[X]
embed.shape

torch.Size([32, 3, 2])

### Reshape the embedding layer

In [31]:
embed = embed.view(-1, 6)
embed.shape

torch.Size([32, 6])

## Hidden Layer

In [35]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)
h = torch.tanh(embed.view(-1, 6) @ W1 + b1)
h.shape

torch.Size([32, 100])

In [37]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
logits = h @ W2 + b2
logits.shape

torch.Size([32, 27])

In [46]:
counts = logits.exp()
prob = counts / counts.sum(dim=1, keepdim=True)
prob.shape

torch.Size([32, 27])

In [48]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.9015)

## Full Forward Pass

In [49]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [50]:
sum(p.nelement() for p in parameters)

3481

In [51]:
embed = C[X]
h = torch.tanh(embed.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.9015)